# Performing Testing at Each Round of Training 

Use for developing (autoreloads changes made across packages)

In [ ]:
%load_ext autoreload
%autoreload 2

## Start the network
Before running this notebook, start the network with `./scripts/fedbiomed_run network`

## Setting the node up
It is necessary to previously configure a node:
1. `./scripts/fedbiomed_run node add`
  * Select option 2 (default) to add MNIST to the node
  * Confirm default tags by hitting "y" and ENTER
  * Pick the folder where MNIST is downloaded (this is due torch issue https://github.com/pytorch/vision/issues/3549)
  * Data must have been added (if you get a warning saying that data must be unique is because it's been already added)
  
2. Check that your data has been added by executing `./scripts/fedbiomed_run node list`
3. Run the node using `./scripts/fedbiomed_run node run`. Wait until you get `Starting task manager`. it means you are online.

# 1. Testing and training a Pytorch model using a default metric

Declare a torch.nn MyTrainingPlan class to send for training on the node. Here the model is trained on the **MNIST dataset** for classification. For testing, we will be using the **F1-score** metric.

Testing will be performed on both **local updates and global updates**.

In [1]:
import torch
import torch.nn as nn
from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.data import DataManager
from torchvision import datasets, transforms

# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlan(TorchTrainingPlan):
    def __init__(self, model_args: dict = {}):
        super(MyTrainingPlan, self).__init__(model_args)
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch DataLoader classes
        # Since we will train on MNIST, we need datasets and transform from torchvision
        deps = ["from torchvision import datasets, transforms"]
        
        self.add_dependency(deps)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        
        
        output = F.log_softmax(x, dim=1)
        return output

    def training_data(self, batch_size = 48):
        # Custom torch Dataloader for MNIST data
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset1 = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        return DataManager(dataset=dataset1, **train_kwargs)
    
    def training_step(self, data, target):
        output = self.forward(data)
        loss   = torch.nn.functional.nll_loss(output, target)
        return loss


This group of arguments correspond respectively:
* `model_args`: a dictionary with the arguments related to the model (e.g. number of layers, features, etc.). This will be passed to the model class on the node side.
* `training_args`: a dictionary containing the arguments for the training routine (e.g. batch size, learning rate, epochs, etc.). This will be passed to the routine on the node side.

**NOTE:** typos and/or lack of positional (required) arguments will raise error. 🤓

In [2]:
from fedbiomed.common.metrics import MetricTypes

model_args = {}

training_args = {
    'batch_size': 48, 
    'lr': 1e-3, 
    'epochs': 1, 
    'dry_run': False,  
    'batch_maxnum': 100, # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
  
}

## Declare and run the experiment

- search nodes serving data for these `tags`, optionally filter on a list of node ID with `nodes`
- run a round of local training on nodes with model defined in `model_path` + federation with `aggregator`
- run for `round_limit` rounds, applying the `node_selection_strategy` between the rounds

In [3]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#MNIST', '#dataset']
rounds = 1

exp = Experiment(tags=tags,
                 model_args=model_args,
                 model_class=MyTrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None,
                tensorboard=True)


exp.set_test_ratio(0.1)
exp.set_test_on_local_updates(True)
exp.set_test_on_global_updates(True)
exp.set_test_metric(MetricTypes.F1_SCORE, zero_division='warn')

2022-03-28 13:34:30,035 fedbiomed INFO - Component environment:
2022-03-28 13:34:30,036 fedbiomed INFO - type = ComponentType.RESEARCHER
2022-03-28 13:34:30,201 fedbiomed INFO - Messaging researcher_0d1043ef-15a8-4302-a2a0-f3874aa0aa38 successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x7f991dc0d460>
2022-03-28 13:34:30,228 fedbiomed INFO - Searching dataset with data tags: ['#MNIST', '#dataset'] for all nodes
2022-03-28 13:34:40,247 fedbiomed INFO - Node selected for training -> node_77dadfb2-a01c-4f37-9699-82c552088866
2022-03-28 13:34:40,249 fedbiomed INFO - Node selected for training -> node_7321fdef-ae0f-4548-a5a8-1eadf9043b31
2022-03-28 13:34:40,254 fedbiomed INFO - Checking data quality of federated datasets...
2022-03-28 13:34:40,288 fedbiomed DEBUG - Model file has been saved: /home/ybouilla/fedbiomed/var/experiments/Experiment_0016/my_model_ff295764-f418-49c9-8060-60b7d63de844.py
2022-03-28 13:34:40,314 fedbiomed DEBUG - up

(<MetricTypes.F1_SCORE: (1, <_MetricCategory.CLASSIFICATION_LABELS: 0>)>,
 {'zero_division': 'warn'})

Launch tensorboard

In [4]:
from fedbiomed.researcher.environ import environ
tensorboard_dir = environ['TENSORBOARD_RESULTS_DIR']

In [5]:
%load_ext tensorboard

In [6]:
tensorboard --logdir "$tensorboard_dir"

Let's start the experiment.

By default, this function doesn't stop until all the `round_limit` rounds are done for all the nodes

In [7]:
exp.run()

2022-03-28 13:34:41,546 fedbiomed INFO - Sampled nodes in round 0 ['node_77dadfb2-a01c-4f37-9699-82c552088866', 'node_7321fdef-ae0f-4548-a5a8-1eadf9043b31']
2022-03-28 13:34:41,548 fedbiomed INFO - Sending request 
					 To: node_77dadfb2-a01c-4f37-9699-82c552088866 
					 Reqeust: : Perform training with the arguments: {'researcher_id': 'researcher_0d1043ef-15a8-4302-a2a0-f3874aa0aa38', 'job_id': '871dc3f9-bce1-4535-af24-c0890e8a551f', 'training_args': {'test_ratio': 0.1, 'test_on_local_updates': True, 'test_on_global_updates': True, 'test_metric': <MetricTypes.F1_SCORE: (1, <_MetricCategory.CLASSIFICATION_LABELS: 0>)>, 'test_metric_args': {'zero_division': 'warn'}, 'batch_size': 48, 'lr': 0.001, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 100}, 'training': True, 'model_args': {}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/03/28/my_model_ff295764-f418-49c9-8060-60b7d63de844.py', 'params_url': 'http://localhost:8844/media/uploads/2022/03/28/aggregated

2022-03-28 13:34:57,478 fedbiomed INFO - INFO
					 NODE node_77dadfb2-a01c-4f37-9699-82c552088866
					 MESSAGE: Actual/True values (y_true) has more than two levels, using multiclass `weighted` calculation for the metric F1_SCORE
-----------------------------------------------------------------
2022-03-28 13:34:57,483 fedbiomed INFO - TESTING ON LOCAL UPDATES 
					 NODE_ID: node_77dadfb2-a01c-4f37-9699-82c552088866 
					 Completed: 6000/6000 (100%) 
 					 F1_SCORE: 0.934793 
					 ---------
2022-03-28 13:34:57,696 fedbiomed INFO - INFO
					 NODE node_77dadfb2-a01c-4f37-9699-82c552088866
					 MESSAGE: results uploaded successfully 
-----------------------------------------------------------------
2022-03-28 13:34:58,588 fedbiomed INFO - INFO
					 NODE node_7321fdef-ae0f-4548-a5a8-1eadf9043b31
					 MESSAGE: Actual/True values (y_true) has more than two levels, using multiclass `weighted` calculation for the metric F1_SCORE
--------------------------------------------------------

1

In [12]:
exp._round_current, exp._round_limit

(3, 3)

Local training results for each round and each node are available via `exp.training_replies()` (index 0 to (`rounds` - 1) ).

For example you can view the training results for the last round below.

Different timings (in seconds) are reported for each dataset of a node participating in a round :
- `rtime_training` real time (clock time) spent in the training function on the node
- `ptime_training` process time (user and system CPU) spent in the training function on the node
- `rtime_total` real time (clock time) spent in the researcher between sending the request and handling the response, at the `Job()` layer

In [6]:
print("\nList the training rounds : ", exp.training_replies().keys())

print("\nList the nodes for the last training round and their timings : ")
round_data = exp.training_replies()[rounds - 1].data()
for c in range(len(round_data)):
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = round_data[c]['node_id'],
        rtraining = round_data[c]['timing']['rtime_training'],
        ptraining = round_data[c]['timing']['ptime_training'],
        rtotal = round_data[c]['timing']['rtime_total']))
print('\n')
    
exp.training_replies()[rounds - 1].dataframe()


List the training rounds :  dict_keys([0, 1, 2, 3, 4])

List the nodes for the last training round and their timings : 
	- node_e2fb6eb5-2e15-443a-b6b5-a5da28a45524 :    
		rtime_training=4.34 seconds    
		ptime_training=17.23 seconds    
		rtime_total=25.03 seconds




,success,msg,dataset_id,node_id,params_path,params,timing
0,True,,dataset_d12c210c-31ff-434f-b024-3da80d73c369,node_e2fb6eb5-2e15-443a-b6b5-a5da28a45524,/home/ybouilla/fedbiomed/var/experiments/Exper...,"{'conv1.weight': [[tensor([[-0.3393, -0.1751, ...","{'rtime_training': 4.337263318000623, 'ptime_t..."


Federated parameters for each round are available via `exp.aggregated_params()` (index 0 to (`rounds` - 1) ).

For example you can view the federated parameters for the last round of the experiment :

In [7]:
print("\nList the training rounds : ", exp.aggregated_params().keys())

print("\nAccess the federated params for the last training round :")
print("\t- params_path: ", exp.aggregated_params()[rounds - 1]['params_path'])
print("\t- parameter data: ", exp.aggregated_params()[rounds - 1]['params'].keys())



List the training rounds :  dict_keys([0, 1, 2, 3])

Access the federated params for the last training round :
	- params_path:  /home/ybouilla/fedbiomed/var/experiments/Experiment_0007/aggregated_params_5c4d0680-c370-435f-ba8a-4367c6c122ca.pt
	- parameter data:  odict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias'])


**List of Default metrics** 
(please check Scikit - learn for further details)

In [14]:
MetricTypes.get_all_metrics()

['ACCURACY',
 'F1_SCORE',
 'PRECISION',
 'AVG_PRECISION',
 'RECALL',
 'ROC_AUC',
 'MEAN_SQUARE_ERROR',
 'MEAN_ABSOLUTE_ERROR',
 'EXPLAINED_VARIANCE']



## 2. Training and Testing with sklearn Perceptron model


Now we will use the testing facility on Skelearn training plan

In [10]:

from fedbiomed.common.training_plans import SGDSkLearnModel
from fedbiomed.common.data import DataManager
import numpy as np


class SkLearnClassifierTrainingPlan(SGDSkLearnModel):
    def __init__(self, model_args):
        super(SkLearnClassifierTrainingPlan,self).__init__(model_args)
        self.add_dependency(['import torch',
                            "from sklearn.linear_model import Perceptron",
                            "from torchvision import datasets, transforms",
                           "from torch.utils.data import DataLoader"])
    
    
    def training_data(self):
        # Custom torch Dataloader for MNIST data
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        
        train_kwargs = {'batch_size': 500, 'shuffle': True}  # number of data passed to classifier
        X_train = dataset.data.numpy()
        X_train = X_train.reshape(-1, 28*28)
        Y_train = dataset.targets.numpy()
        
        return DataManager(dataset=X_train,target=Y_train)

In [11]:
model_args = { 'max_iter':1000,
              'tol': 1e-4 ,
              'model': 'Perceptron' ,
              'n_features': 28*28,
              'n_classes' : 10,
              'eta0':1e-6,
             'random_state':1234,
             'alpha':0.1 }

training_args = {
    'epochs': 1, 
    'test_metric': "PRECISION",
    'test_ratio': .2,
   'test_on_global_updates': True,
    'test_metric_args': {'average': 'macro'}
}

In [18]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#MNIST', '#dataset']
rounds = 50

# select nodes participing to this experiment
exp = Experiment(tags=tags,
                 model_args=model_args,
                 model_class=SkLearnClassifierTrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None, 
                tensorboard=True)


2022-03-28 13:56:03,988 fedbiomed INFO - Searching dataset with data tags: ['#MNIST', '#dataset'] for all nodes
2022-03-28 13:56:14,011 fedbiomed INFO - Node selected for training -> node_77dadfb2-a01c-4f37-9699-82c552088866
2022-03-28 13:56:14,013 fedbiomed INFO - Node selected for training -> node_7321fdef-ae0f-4548-a5a8-1eadf9043b31
2022-03-28 13:56:14,017 fedbiomed INFO - Checking data quality of federated datasets...
2022-03-28 13:56:14,026 fedbiomed CRITICAL - Fed-BioMed stopped due to unknown error:
__init__() missing 1 required positional argument: 'model_args'



--------------------
Fed-BioMed researcher stopped due to unknown error:
__init__() missing 1 required positional argument: 'model_args'
More details in the backtrace extract below
--------------------
Traceback (most recent call last):
  File "/home/ybouilla/fedbiomed/fedbiomed/researcher/experiment.py", line 58, in payload
    ret = function(*args, **kwargs)
  File "/home/ybouilla/fedbiomed/fedbiomed/researcher/experiment.py", line 1249, in set_job
    self._job = Job(reqs=self._reqs,
  File "/home/ybouilla/fedbiomed/fedbiomed/researcher/job.py", line 133, in __init__
    self.model_instance = model()  # contains TrainingPlan
TypeError: __init__() missing 1 required positional argument: 'model_args'
--------------------


In [13]:
exp.run(rounds=1, increase=True)

2022-03-28 13:48:18,074 fedbiomed INFO - Sampled nodes in round 0 ['node_77dadfb2-a01c-4f37-9699-82c552088866', 'node_7321fdef-ae0f-4548-a5a8-1eadf9043b31']
2022-03-28 13:48:18,076 fedbiomed INFO - Sending request 
					 To: node_77dadfb2-a01c-4f37-9699-82c552088866 
					 Reqeust: : Perform training with the arguments: {'researcher_id': 'researcher_0d1043ef-15a8-4302-a2a0-f3874aa0aa38', 'job_id': 'a0e91bc4-7db5-427a-b81b-2c025da7ca51', 'training_args': {'test_ratio': 0.2, 'test_on_local_updates': False, 'test_on_global_updates': True, 'test_metric': 'PRECISION', 'test_metric_args': {'average': 'macro'}, 'epochs': 1}, 'training': True, 'model_args': {'max_iter': 1000, 'tol': 0.0001, 'model': 'Perceptron', 'n_features': 784, 'n_classes': 10, 'eta0': 1e-06, 'random_state': 1234, 'alpha': 0.1, 'verbose': 1}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/03/28/my_model_dc8ba3fd-0ef2-43ca-a335-b24f40baa4a5.py', 'params_url': 'http://localhost:8844/media/uploads/

2022-03-28 13:48:19,395 fedbiomed INFO - INFO
					 NODE node_7321fdef-ae0f-4548-a5a8-1eadf9043b31
					 MESSAGE: Train Epoch: 0 [Batch All Samples]	Loss: 0.073247
-----------------------------------------------------------------
2022-03-28 13:48:19,397 fedbiomed INFO - INFO
					 NODE node_7321fdef-ae0f-4548-a5a8-1eadf9043b31
					 MESSAGE: Train Epoch: 0 [Batch All Samples]	Loss: 0.127751
-----------------------------------------------------------------
2022-03-28 13:48:19,399 fedbiomed INFO - INFO
					 NODE node_7321fdef-ae0f-4548-a5a8-1eadf9043b31
					 MESSAGE: Train Epoch: 0 [Batch All Samples]	Loss: 0.058865
-----------------------------------------------------------------
2022-03-28 13:48:19,400 fedbiomed INFO - INFO
					 NODE node_7321fdef-ae0f-4548-a5a8-1eadf9043b31
					 MESSAGE: Train Epoch: 0 [Batch All Samples]	Loss: 0.060112
-----------------------------------------------------------------
2022-03-28 13:48:19,403 fedbiomed INFO - INFO
					 NODE node_7321fdef-ae0f-4548

1

Feel free to run other sample notebooks or try your own models :D

# 3. Testing facility using your own testing metric

If the user wants to define its own testing metric, he can do so by defining the `testing_step` method in the Training plan. 

`testing_step` is defined the same way as `trainig_step`:

User has to:
- predict classes or probabilities from model
- compute a scalar or a list of scalars

list of scalars will be seen as the output of several metrics

In [15]:
import torch
import torch.nn as nn
from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.data import DataManager
from torchvision import datasets, transforms

# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlanCM(TorchTrainingPlan):
    def __init__(self, model_args: dict = {}):
        super(MyTrainingPlanCM, self).__init__(model_args)
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch DataLoader classes
        # Since we will train on MNIST, we need datasets and transform from torchvision
        deps = ["from torchvision import datasets, transforms"]
        
        self.add_dependency(deps)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        
        
        output = F.log_softmax(x, dim=1)
        return output

    def training_data(self, batch_size = 48):
        # Custom torch Dataloader for MNIST data
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset1 = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        return DataManager(dataset=dataset1, **train_kwargs)
    
    def training_step(self, data, target):
        output = self.forward(data)
        loss   = torch.nn.functional.nll_loss(output, target)
        return loss

    def testing_step(self, data, target):
        
        output = self.forward(data)
        loss1   = torch.nn.functional.nll_loss(output, target)
        output = self(data)
        loss2   = torch.nn.functional.nll_loss(output, target)
        return [loss1, loss2]

In [16]:
model_args = {}

training_args = {
    'batch_size': 48, 
    'lr': 1e-3, 
    'epochs': 1, 
    'dry_run': False,  
    'batch_maxnum': 100, # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
    'test_ratio': .3,
    'test_on_local_updates': True, 
    'test_on_global_updates': True
}

In [3]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#MNIST', '#dataset']
rounds = 2

exp = Experiment(tags=tags,
                 model_args=model_args,
                 model_class=MyTrainingPlanCM,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None, 
                tensorboard=True)

2022-03-25 18:30:02,029 fedbiomed INFO - Component environment:
2022-03-25 18:30:02,031 fedbiomed INFO - type = ComponentType.RESEARCHER
2022-03-25 18:30:02,197 fedbiomed INFO - Messaging researcher_34aeb0d0-28b0-4032-968b-00d398e0d668 successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x7f79067b9070>
2022-03-25 18:30:02,229 fedbiomed INFO - Searching dataset with data tags: ['#MNIST', '#dataset'] for all nodes
2022-03-25 18:30:12,242 fedbiomed INFO - Node selected for training -> node_e2fb6eb5-2e15-443a-b6b5-a5da28a45524
2022-03-25 18:30:12,266 fedbiomed DEBUG - Model file has been saved: /home/ybouilla/fedbiomed/var/experiments/Experiment_0012/my_model_b35b47e1-d6df-4d06-8baa-b4314d1c9058.py
2022-03-25 18:30:12,290 fedbiomed DEBUG - upload (HTTP POST request) of file /home/ybouilla/fedbiomed/var/experiments/Experiment_0012/my_model_b35b47e1-d6df-4d06-8baa-b4314d1c9058.py successful, with status code 201
2022-03-25 18:30:12,437 fedbi

In [4]:
exp.run()

2022-03-25 18:30:12,442 fedbiomed INFO - Sampled nodes in round 0 ['node_e2fb6eb5-2e15-443a-b6b5-a5da28a45524']
2022-03-25 18:30:12,444 fedbiomed INFO - Sending request 
					 To: node_e2fb6eb5-2e15-443a-b6b5-a5da28a45524 
					 Reqeust: : Perform training with the arguments: {'researcher_id': 'researcher_34aeb0d0-28b0-4032-968b-00d398e0d668', 'job_id': '29d230e5-1fd0-4d85-946a-996a36c81d78', 'training_args': {'test_ratio': 0.3, 'test_on_local_updates': True, 'test_on_global_updates': True, 'test_metric': None, 'test_metric_args': {}, 'batch_size': 48, 'lr': 0.001, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 100}, 'training': True, 'model_args': {}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/03/25/my_model_b35b47e1-d6df-4d06-8baa-b4314d1c9058.py', 'params_url': 'http://localhost:8844/media/uploads/2022/03/25/aggregated_params_init_6909edde-4cab-45ef-bcba-2a3706676bf9.pt', 'model_class': 'MyTrainingPlanCM', 'training_data': {'node_e2fb6eb5-2e15-443a-b6

2022-03-25 18:31:02,963 fedbiomed INFO - TRAINING 
					 NODE_ID: node_e2fb6eb5-2e15-443a-b6b5-a5da28a45524 
					 Epoch: 1 | Completed: 2880/42000 (7%) 
 					 Loss: 0.428365 
					 ---------
2022-03-25 18:31:03,403 fedbiomed INFO - TRAINING 
					 NODE_ID: node_e2fb6eb5-2e15-443a-b6b5-a5da28a45524 
					 Epoch: 1 | Completed: 3360/42000 (8%) 
 					 Loss: 0.166382 
					 ---------
2022-03-25 18:31:03,837 fedbiomed INFO - TRAINING 
					 NODE_ID: node_e2fb6eb5-2e15-443a-b6b5-a5da28a45524 
					 Epoch: 1 | Completed: 3840/42000 (9%) 
 					 Loss: 0.149749 
					 ---------
2022-03-25 18:31:04,267 fedbiomed INFO - TRAINING 
					 NODE_ID: node_e2fb6eb5-2e15-443a-b6b5-a5da28a45524 
					 Epoch: 1 | Completed: 4320/42000 (10%) 
 					 Loss: 0.241099 
					 ---------
2022-03-25 18:31:16,639 fedbiomed INFO - TESTING ON GLOBAL PARAMETERS 
					 NODE_ID: node_e2fb6eb5-2e15-443a-b6b5-a5da28a45524 
					 Completed: 18000/18000 (100%) 
 					 Custom_1: 0.125541 
					 Custom_2: 0.125541 
					 --

2

In [5]:
exp.run(rounds=1, increase=True)

2022-03-25 18:31:42,955 fedbiomed DEBUG - Auto increasing total rounds for experiment from 2 to 3
2022-03-25 18:31:42,958 fedbiomed INFO - Sampled nodes in round 2 ['node_e2fb6eb5-2e15-443a-b6b5-a5da28a45524']
2022-03-25 18:31:42,960 fedbiomed INFO - Sending request 
					 To: node_e2fb6eb5-2e15-443a-b6b5-a5da28a45524 
					 Reqeust: : Perform training with the arguments: {'researcher_id': 'researcher_34aeb0d0-28b0-4032-968b-00d398e0d668', 'job_id': '29d230e5-1fd0-4d85-946a-996a36c81d78', 'training_args': {'test_ratio': 0.3, 'test_on_local_updates': True, 'test_on_global_updates': True, 'test_metric': None, 'test_metric_args': {}, 'batch_size': 48, 'lr': 0.001, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 100}, 'training': True, 'model_args': {}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/03/25/my_model_b35b47e1-d6df-4d06-8baa-b4314d1c9058.py', 'params_url': 'http://localhost:8844/media/uploads/2022/03/25/aggregated_params_71d82da7-0423-46b4-83f6-b197b

1